In [29]:
#!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client spotipy

In [30]:
# CONFIGURABLE PARAMETERS

YOUTUBE_PLAYLIST_TITLE = "test_spotify2youtube"

# Replace with the Spotify playlist ID or URI
SPOTIFY_PLAYLIST_ID = "https://open.spotify.com/playlist/3pm336mFUMDWYAAUvfeGFQ?si=58756150192b4017"
# Replace with your Spotify API credentials: https://developer.spotify.com/
SPOTIFY_CLIENT_ID = "x"
SPOTIFY_CLIENT_SECRET = "x"

YOUTUBE_QUERY_SUFFIX = 'Karaoke'

In [31]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Scopes for accessing YouTube Data API
SCOPES = ['https://www.googleapis.com/auth/youtube']

def authenticate_youtube():
    """Authenticate with the YouTube API using OAuth 2.0. Download the client_secrets file from Youtube Data API v3 in Google Console"""
    flow = InstalledAppFlow.from_client_secrets_file(
        'client_secret.json', SCOPES)
    credentials = flow.run_local_server()
    return build('youtube', 'v3', credentials=credentials)

def search_video(youtube, query):
    """Search for a video on YouTube."""
    request = youtube.search().list(
        part='snippet',
        q=query,
        type='video',
        maxResults=1
    )
    response = request.execute()
    return response['items'][0]['id']['videoId'] if response['items'] else None

def add_to_playlist(youtube, playlist_id, video_id):
    """Add a video to a YouTube playlist."""
    request = youtube.playlistItems().insert(
        part='snippet',
        body={
            'snippet': {
                'playlistId': playlist_id,
                'resourceId': {
                    'kind': 'youtube#video',
                    'videoId': video_id
                }
            }
        }
    )
    response = request.execute()
    print(f"Added video {video_id} to playlist {playlist_id}")
    return response

def create_playlist(youtube, title, description=""):
    """Create a new playlist."""
    request = youtube.playlists().insert(
        part='snippet,status',
        body={
            'snippet': {
                'title': title,
                'description': description
            },
            'status': {
                'privacyStatus': 'private'  # You can set 'public' or 'unlisted' if needed
            }
        }
    )
    response = request.execute()
    print(f"Playlist '{title}' created with ID: {response['id']}")
    return response['id']

def get_or_create_playlist(youtube, playlist_title):
    """Check if a playlist exists; create it if not."""
    # Search for the playlist
    request = youtube.playlists().list(
        part='snippet',
        mine=True,  # Ensures you're searching only the authenticated user's playlists
        maxResults=50
    )
    response = request.execute()

    # Check if the playlist exists
    for playlist in response.get('items', []):
        if playlist['snippet']['title'] == playlist_title:
            print(f"Found existing playlist: {playlist_title}")
            return playlist['id']

    # If not found, create the playlist
    print(f"Playlist '{playlist_title}' not found. Creating it...")
    return create_playlist(youtube, playlist_title)



In [32]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def authenticate_spotify(client_id, client_secret):
    """Authenticate with Spotify API."""
    client_credentials_manager = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_playlist_tracks(sp, playlist_id):
    """Retrieve songs and artist names from a Spotify playlist."""
    tracks = []
    results = sp.playlist_tracks(playlist_id)
    
    while results:
        for item in results['items']:
            track = item['track']
            track_name = track['name']
            artist_names = ', '.join([artist['name'] for artist in track['artists']])
            tracks.append((track_name, artist_names))
        
        # Handle pagination if there are more tracks
        results = sp.next(results) if results['next'] else None

    return tracks

In [33]:
def main():
    # Authenticate and build the YouTube API client
    youtube = authenticate_youtube()
    spotify = authenticate_spotify(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
    tracks = get_playlist_tracks(spotify, SPOTIFY_PLAYLIST_ID)
    
    # Get or create the YouTube playlist
    playlist_id = get_or_create_playlist(youtube, YOUTUBE_PLAYLIST_TITLE)
    
    for track_name, artist_names in tracks:
        # Search for a video
        query = f"{track_name} by {artist_names} {YOUTUBE_QUERY_SUFFIX}"
        video_id = search_video(youtube, query)
        if not video_id:
            print("No video found for the query.")
            return

        # Add the video to the playlist
        try:
            add_to_playlist(youtube, playlist_id, video_id)
        except HttpError as e:
            print(f"An error occurred for {track_name} by {artist_names}: {e}")

In [34]:
if __name__ == '__main__':
    main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=228085352117-otg06thbabbg82i10vp5q5smb67e6vd6.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=C0EE9dQTxFRHuDzXQbAY8vKH8DSDLt&access_type=offline
Found existing playlist: test_spotify2youtube
Added video JWnw-YyIcSY to playlist PL3-O7YCuBe7Z9lqzVf065ZNxXr2JEIA09
Added video jm6LeFub0kM to playlist PL3-O7YCuBe7Z9lqzVf065ZNxXr2JEIA09
Added video w7KHcWe5ACw to playlist PL3-O7YCuBe7Z9lqzVf065ZNxXr2JEIA09
Added video J1wXFl3-ZyI to playlist PL3-O7YCuBe7Z9lqzVf065ZNxXr2JEIA09
Added video FFdUHOhwTFU to playlist PL3-O7YCuBe7Z9lqzVf065ZNxXr2JEIA09
Added video Xc4IAC-UoRM to playlist PL3-O7YCuBe7Z9lqzVf065ZNxXr2JEIA09
Added video vE_SfVPX-bM to playlist PL3-O7YCuBe7Z9lqzVf065ZNxXr2JEIA09
Added video Y0aRpJxVMJ0 to playlist PL3-O7YCuBe7Z9lqzVf065ZNxXr2JEIA09
Added video HbcTxD1PnJw to pla

HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/search?part=snippet&q=%E7%88%B1%E4%BA%BA%E9%94%99%E8%BF%87+by+%E6%97%A0%E5%AD%9F%E5%AD%90+Karaoke&type=video&maxResults=1&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">